In [1]:
import pickle
import pandas as pd
from scipy.sparse import hstack
from scipy.sparse.csr import csr_matrix
import tldextract

In [2]:
with open('chargebee_model_objects_jan5.pkl','r') as f:
    tmp = pickle.load(f)
vectorizer,model,bin_boundaries,bin_labels = tmp['vectorizer'],tmp['model'],tmp['bin_boundaries'],tmp['bin_labels']

### dec 28 list for scoring

In [10]:
df1 = pd.read_excel('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_enrichment/scoring_lists/dec28/Saas_data_for_scoring_output.xls')
df2 = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_enrichment/scoring_lists/dec28/Saas_data_for_scoring_output_all_texts.csv',
                 names = ['website','website_text'])

In [13]:
df1['domain'] = df1['website'].apply(lambda x: tldextract.extract(x).domain)
df2['domain'] = df2['website'].apply(lambda x: tldextract.extract(x).domain)

No handlers could be found for logger "tldextract"


In [14]:
df1.head()

,company_linkedin_url,demo_present,emails,id,login_present,match_texts_test,pricing_present,score,urls,website,domain
0,NaN,True,[u'https://conversionista.se/en/conversion-jam...,850,True,"[((u'Pricing',), u""There were some fabulous su...",True,127.5,[u'https://www.twitter.com/jbmoreno/status/251...,http://www.bryaneisenberg.com,bryaneisenberg
1,NaN,True,[],2180,True,"[((u'trial',), u'Reviewsnap Performance Manage...",True,102.4,"[u'https://www.twitter.com/reviewsnap', u'http...",http://www.reviewsnap.com,reviewsnap
2,NaN,True,[],219,True,"[((u'SaaS',), u""We 'cloudify' over 60 best-of-...",False,74.0,[u'https://www.linkedin.com/company/avanan-inc...,http://www.avanan.com,avanan
3,NaN,True,[],218,True,"[((u'SaaS',), u""We 'cloudify' over 60 best-of-...",False,72.5,[u'https://www.linkedin.com/company/avanan-inc...,http://www.avanan.com,avanan
4,NaN,False,[],1557,True,"[((u'subscription',), u'Cancel Anytime\nCancel...",False,60.3,"[u'https://www.facebook.com/lootcrate', u'http...",http://www.lootcrate.com,lootcrate


In [15]:
df2.head()

,website,website_text,domain
0,http://www.101edu.co,101 - Mobile-first learning tools built specif...,101edu
1,http://www.1776.vc,Where Innovation Begins | 1776 Startup Incubat...,1776
2,http://www.1871.com,1871 - Chicago's Technology & Entrepreneurship...,1871
3,http://www.sero.uy,"$ERO, la cuenta para los que no tienen cuenta ...",sero
4,http://www.tapslash.com,NaN,tapslash


In [18]:
df = pd.merge(df1,df2,on='domain')

In [19]:
df.head()

,company_linkedin_url,demo_present,emails,id,login_present,match_texts_test,pricing_present,score,urls,website_x,domain,website_y,website_text
0,NaN,True,[u'https://conversionista.se/en/conversion-jam...,850,True,"[((u'Pricing',), u""There were some fabulous su...",True,127.5,[u'https://www.twitter.com/jbmoreno/status/251...,http://www.bryaneisenberg.com,bryaneisenberg,http://www.bryaneisenberg.com,Bryan & Jeffrey Eisenberg - Professional Speak...
1,NaN,True,[],2180,True,"[((u'trial',), u'Reviewsnap Performance Manage...",True,102.4,"[u'https://www.twitter.com/reviewsnap', u'http...",http://www.reviewsnap.com,reviewsnap,http://www.reviewsnap.com,Employee Performance Management Software Syste...
2,NaN,True,[],219,True,"[((u'SaaS',), u""We 'cloudify' over 60 best-of-...",False,74.0,[u'https://www.linkedin.com/company/avanan-inc...,http://www.avanan.com,avanan,http://www.avanan.com,Avanan - The Cloud Security Platform\n+1.855.5...
3,NaN,True,[],219,True,"[((u'SaaS',), u""We 'cloudify' over 60 best-of-...",False,74.0,[u'https://www.linkedin.com/company/avanan-inc...,http://www.avanan.com,avanan,http://www.avanan.com,Avanan - The Cloud Security Platform\n+1.855.5...
4,NaN,True,[],218,True,"[((u'SaaS',), u""We 'cloudify' over 60 best-of-...",False,72.5,[u'https://www.linkedin.com/company/avanan-inc...,http://www.avanan.com,avanan,http://www.avanan.com,Avanan - The Cloud Security Platform\n+1.855.5...


In [21]:
tfidf = vectorizer.transform(df['website_text'].fillna(''))

In [23]:
tfidf_n_vars = hstack([tfidf,
        csr_matrix(df[['demo_present','login_present','pricing_present']].values.astype(float))])
tfidf_n_vars.shape

(2322, 695)

In [24]:
score = model.predict_proba(tfidf_n_vars)

In [32]:
df['saas_subscription_score'] = score[:,1]
df['saas_subscription_score_bin'] = pd.cut(df['saas_subscription_score'],bin_boundaries,labels=bin_labels)

In [33]:
df_out = df[['id','website_x','domain','login_present','pricing_present','demo_present','score','saas_subscription_score','saas_subscription_score_bin']]
df_out.columns = ['id','website','domain','login_present','pricing_present','demo_present','raw_score','saas_subscription_final_score','saas_subscription_score_bin']

In [34]:
df_out.to_excel('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_enrichment/scoring_lists/dec28/Saas_data_for_scoring_with_scores.xls',index=False)

In [35]:
df['saas_subscription_score_bin'].value_counts()

1    357
5    300
4    267
3    253
6    232
7    217
0    192
2    191
9    187
8    126
Name: saas_subscription_score_bin, dtype: int64

In [49]:
#second list
df1 = pd.read_excel('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_enrichment/scoring_lists/dec28/Saas_data_for_scoring_1_output.xls')
df2 = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_enrichment/scoring_lists/dec28/Saas_data_for_scoring_1_output_all_texts.csv',
                 names = ['website','website_text'])

In [50]:
df1['domain'] = df1['website'].apply(lambda x: tldextract.extract(x).domain)
df2['domain'] = df2['website'].apply(lambda x: tldextract.extract(x).domain)

In [51]:
df = pd.merge(df1,df2,on='domain')

In [52]:
tfidf = vectorizer.transform(df['website_text'].fillna(''))
tfidf_n_vars = hstack([tfidf,
        csr_matrix(df[['demo_present','login_present','pricing_present']].values.astype(float))])
tfidf_n_vars.shape

(127, 695)

In [53]:
score = model.predict_proba(tfidf_n_vars)
df['saas_subscription_score'] = score[:,1]
df['saas_subscription_score_bin'] = pd.cut(df['saas_subscription_score'],bin_boundaries,labels=bin_labels)

In [54]:
df_out = df[['id','website_x','domain','login_present','pricing_present','demo_present','score','saas_subscription_score','saas_subscription_score_bin']]
df_out.columns = ['id','website','domain','login_present','pricing_present','demo_present','raw_score','saas_subscription_final_score','saas_subscription_score_bin']

In [55]:
df_out.to_excel('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_enrichment/scoring_lists/dec28/Saas_data_for_scoring_1_with_scores.xls',index=False)

In [60]:
pd.DataFrame(df['saas_subscription_score_bin'].value_counts()).sort()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(....) is deprecated, use sort_index(.....)
  if __name__ == '__main__':


,saas_subscription_score_bin
0,9
1,23
2,6
3,18
4,17
5,16
6,13
7,13
8,0
9,12


### jan2 list

In [7]:
tmp1 = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_works/nlp-intelligence/scoring_lists/jan02_shrikanth/SFCustomers_Sample_out.csv')
tmp2 = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_works/nlp-intelligence/scoring_lists/jan02_shrikanth/SFCustomers_Sample1_out.csv')
tmp11 = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_works/nlp-intelligence/scoring_lists/jan02_shrikanth/SFCustomers_Sample_out_all_texts.csv',names=['website','website_text'])
tmp21 = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_works/nlp-intelligence/scoring_lists/jan02_shrikanth/SFCustomers_Sample1_out_all_texts.csv',names=['website','website_text'])

In [11]:
tmp1_1 = pd.merge(tmp1,tmp11,on='website')
tmp2_1 = pd.merge(tmp2,tmp21,on='website')

In [16]:
tmp = pd.concat([tmp1_1,tmp2_1])
tmp.shape

(64, 11)

In [17]:
tfidf = vectorizer.transform(tmp['website_text'].fillna(''))
tfidf_n_vars = hstack([tfidf,
        csr_matrix(tmp[['demo_present','login_present','pricing_present']].values.astype(float))])
tfidf_n_vars.shape

(64, 695)

In [19]:
score = model.predict_proba(tfidf_n_vars)
tmp['saas_subscription_score'] = score[:,1]
tmp['saas_subscription_score_bin'] = pd.cut(tmp['saas_subscription_score'],bin_boundaries,labels=bin_labels)

In [20]:
pd.DataFrame(tmp['saas_subscription_score_bin'].value_counts()).sort()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(....) is deprecated, use sort_index(.....)
  if __name__ == '__main__':


,saas_subscription_score_bin
0,3
1,9
2,7
3,4
4,12
5,13
6,7
7,5
8,3
9,1


In [21]:
tmp.head()

,company_linkedin_url,demo_present,emails,id,login_present,match_texts_test,pricing_present,score,urls,website,website_text,saas_subscription_score,saas_subscription_score_bin
0,NaN,False,[u'mailto:LetUsHelpU@Cigna.com'],25,True,"[((u'Plans', u'Plans', u'Plans', u'Plans'), u'...",True,309.2,"[u'http://www.facebook.com/cigna', u'http://ww...",http://www.cigna.com,Cigna Health Insurance | Global Health Service...,0.391659,4
1,NaN,False,[],63,False,"[((u'Plans', u'Plans'), u'Log In\nLog In to my...",True,79.9,"[u'https://www.twitter.com/myUHC', u'https://w...",http://www.uhc.com,Health Insurance Plans for Individuals & Famil...,0.412283,4
2,NaN,False,[u'mailto:custsvc@elavon.com?subject=Elavon Cu...,73,False,"[((u'payment',), u'Our partnership model is de...",True,53.3,[],http://www.elavon.com,Merchant Services | Elavon\nCookie Policy\nWhe...,0.668152,6
3,NaN,False,"[u'mailto:SI.DRG@uhcsr.com', u'mailto:eftenrol...",60,True,"[((u'payment',), u'In addition to receiving EF...",False,21.9,"[u'http://www.facebook.com/changehealthcare', ...",http://www.uhcsr.com,Student Health Insurance - College Student Ins...,0.632146,6
4,NaN,False,[],59,True,"[((u'Payment',), u'AAPM: Advanced Alternative ...",True,17.8,[u'http://facebook.com/sharer.php?u=http://www...,http://www.optum.com,Health Services and Innovation Company\nSkip t...,0.462763,5


In [24]:
tmp[['id','website','login_present','pricing_present','demo_present','score','saas_subscription_score',
     'saas_subscription_score_bin']].to_excel(
    '/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_works/nlp-intelligence/scoring_lists/jan02_shrikanth/SFCustomers_Sample_scores.xlsx',
index=False)

### october list

In [55]:
df1 = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_works/nlp-intelligence/scoring_lists/oct_list/OCT_2016_Subscriptions_for_scoring_crawler_output.csv')
df2 = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_works/nlp-intelligence/scoring_lists/oct_list/OCT_2016_Subscriptions_for_scoring_crawler_output_1.csv')
df1_text = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_works/nlp-intelligence/scoring_lists/oct_list/OCT_2016_Subscriptions_for_scoring_crawler_output_all_texts.csv',names=['website','website_text'])
df2_text = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_works/nlp-intelligence/scoring_lists/oct_list/OCT_2016_Subscriptions_for_scoring_crawler_output_1_all_texts.csv',names=['website','website_text'])

In [56]:
df1['domain'] = df1['website'].apply(lambda x: tldextract.extract(x).domain)
df2['domain'] = df2['website'].apply(lambda x: tldextract.extract(x).domain)
df1_text['domain'] = df1_text['website'].apply(lambda x: tldextract.extract(x).domain)
df2_text['domain'] = df2_text['website'].apply(lambda x: tldextract.extract(x).domain)

In [57]:
df2_text[df2_text['domain']=='flexretentalsolutions']

,website,website_text,domain
390,http://www.flexretentalsolutions.com,%htmlDTD;\n%netErrorDTD;\nCheck the address fo...,flexretentalsolutions


In [10]:
df1.shape,df1_text.shape,df2.shape,df2_text.shape

((471, 10), (471, 3), (794, 10), (794, 3))

In [58]:
df1 = pd.merge(df1,df1_text,on='domain')
df2 = pd.merge(df2,df2_text,on='domain')
df1.shape,df1_text.shape,df2.shape,df2_text.shape

((533, 12), (471, 3), (890, 12), (794, 3))

In [59]:
df1 = df1.drop_duplicates('domain')
df2 = df2.drop_duplicates('domain')
df1.shape,df2.shape

((450, 12), (759, 12))

In [60]:
# df2 has all data
tmp = pd.merge(df1,df2,how='left',on='domain')
tmp[pd.isnull(tmp['login_present_y'])]

,company_linkedin_url_x,demo_present_x,emails_x,id_x,login_present_x,pricing_present_x,score_x,urls_x,website_x_x,domain,...,demo_present_y,emails_y,id_y,login_present_y,pricing_present_y,score_y,urls_y,website_x_y,website_y_y,website_text_y
439,NaN,False,[],846,False,False,0.0,[],http://www.sazedata.com,sazedata,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df = pd.concat([df2,df1[df1['domain']=='sazedata']])
df.shape,df1.shape,df2.shape

((760, 12), (450, 12), (759, 12))

In [62]:
df = df.drop_duplicates('domain')
df.shape

(760, 12)

In [63]:
df.head()

,company_linkedin_url,demo_present,emails,id,login_present,pricing_present,score,urls,website_x,domain,website_y,website_text
0,NaN,False,[u'mailto:info@planhub.ca'],948,False,False,5968.4,"[u'https://www.twitter.com/PlanHub_ca', u'http...",http://www.planhub.ca,planhub,http://www.planhub.ca,"<img height=""1"" width=""1"" alt="""" style=""displa..."
1,NaN,False,"[u'mailto:legal@checkout.com', u'mailto:sales@...",743,True,True,85.1,[u'https://www.facebook.com/sharer/sharer.php?...,http://www.checkout.com,checkout,http://www.checkout.com,"Checkout.com - Online Payments\n<img src=""http..."
5,NaN,True,"[u'mailto:Accounts@deconetwork.com', u'mailto:...",436,True,True,80.4,"[u'http://www.twitter.com/deconetwork', u'http...",http://www.deconetwork.com,deconetwork,http://www.deconetwork.com,Online T-Shirt Designer and Business Software ...
6,NaN,False,[],374,True,True,76.6,"[u'http://www.twitter.com/legalshield', u'http...",http://www.KeAndrea.com,KeAndrea,http://www.KeAndrea.com,"Associate Page | LegalShield\n<iframe src=""//w..."
7,NaN,False,[u'mailto:sales@hiconversion.com?subject=Custo...,402,True,True,68.5,"[u'http://www.twitter.com/HiConversion', u'htt...",http://www.hiconversion.com,hiconversion,http://www.hiconversion.com,­\nHiConversion | Customer Experience Analytic...


In [65]:
df[df['website_x'].apply(lambda x: tldextract.extract(x).domain) != df['domain']]

,company_linkedin_url,demo_present,emails,id,login_present,pricing_present,score,urls,website_x,domain,website_y,website_text


In [66]:
tfidf = vectorizer.transform(df['website_text'].fillna(''))
#new variables (not used currently in the final model)
reg_text = r'(([0-9]|\$|\b|/)per|/)(\b| |\n|\r){0,2}(month|year|\bmo)\b'
df['per_month_present'] = df['website_text'].fillna('').str.contains(reg_text,case=False)
#coming soon
df['coming_soon'] = df['website_text'].fillna('').str.contains(r'coming soon',case=False)

tfidf_n_vars = hstack([tfidf,
        csr_matrix(df[['demo_present','login_present','pricing_present','per_month_present','coming_soon']].values.astype(float))])
tfidf_n_vars.shape

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


(760, 714)

In [67]:
score = model.predict_proba(tfidf_n_vars)
df['saas_subscription_score'] = score[:,1]
df['saas_subscription_score_bin'] = pd.cut(df['saas_subscription_score'],bin_boundaries,labels=bin_labels)

In [68]:
df[['id','website_x','login_present','pricing_present','demo_present','score','saas_subscription_score',
     'saas_subscription_score_bin']].to_excel(
    '/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_works/nlp-intelligence/scoring_lists/oct_list/OCT 2016 - Subscriptions - for processing_scores.xlsx',
index=False)

In [44]:
[vectorizer.get_feature_names()[ind] for ind in np.where(tfidf.toarray()[632,:]>0)[0]]

['accept',
 'access',
 'account',
 'across',
 'add',
 'age',
 'amount',
 'api',
 'app',
 'blog',
 'browser',
 'built',
 'call',
 'card',
 'center',
 'ceo',
 'chat',
 'co',
 'code',
 'collect',
 'contact',
 'control',
 'credit',
 'dashboard',
 'data',
 'date',
 'demo',
 'digit',
 'done',
 'email',
 'enter',
 'extra',
 'faq',
 'fast',
 'fill',
 'find',
 'first',
 'focus',
 'form',
 'founder',
 'free',
 'full',
 'global',
 'go',
 'great',
 'head',
 'help',
 'high',
 'id',
 'internet',
 'item',
 'join',
 'learn',
 'level',
 'line',
 'live',
 'log',
 'long',
 'major',
 'make',
 'money',
 'must',
 'need',
 'net',
 'never',
 'new',
 'next',
 'number',
 'one',
 'order',
 'page',
 'part',
 'partner',
 'password',
 'pay',
 'payment',
 'platform',
 'popular',
 'process',
 'product',
 'real',
 'safe',
 'save',
 'say',
 'seamless',
 'send',
 'sign',
 'space',
 'spend',
 'standard',
 'start',
 'success',
 'support',
 'system',
 'team',
 'test',
 'time',
 'via',
 'view',
 'visit',
 'way',
 'week',
 '

In [50]:
df2[df2['id']==1106]

,company_linkedin_url,demo_present,emails,id,login_present,pricing_present,score,urls,website_x,domain,website_y,website_text
760,NaN,False,[],1106,False,False,0.0,[],http://www.flexretentalsolutions.com,flexretentalsolutions,http://www.razorpay.com,Razorpay · Payment Gateway Solution for India ...
